In [18]:
import pandas as pd
from scipy import stats

In [17]:
# CARREGAR DADOS
df = pd.read_csv('/content/marketing_AB.csv')
GRUPO, CONVERSAO = 'test group', 'converted'

In [16]:
print(f"Total de usuários: {len(df):,}")
print(f"Grupos:\n{df[GRUPO].value_counts()}")
print(f"Conversões:\n{df[CONVERSAO].value_counts()}")

Total de usuários: 588,101
Grupos:
test group
ad     564577
psa     23524
Name: count, dtype: int64
Conversões:
converted
False    573258
True      14843
Name: count, dtype: int64


In [15]:
dados = df.groupby(GRUPO)[CONVERSAO].agg(['count', 'sum', 'mean']).rename(columns={'count': 'usuarios', 'sum': 'conversoes', 'mean': 'taxa'})
print("\nResultados por grupo:")
print(dados)


Resultados por grupo:
            usuarios  conversoes      taxa
test group                                
ad            564577       14423  0.025547
psa            23524         420  0.017854


In [14]:
grupos = list(dados.index)
if len(grupos) >= 2:
    grupo1, grupo2 = grupos[0], grupos[1]
    n1, conv1, taxa1 = dados.loc[grupo1, ['usuarios', 'conversoes', 'taxa']]
    n2, conv2, taxa2 = dados.loc[grupo2, ['usuarios', 'conversoes', 'taxa']]

    tabela = [[conv1, n1-conv1], [conv2, n2-conv2]]
    chi2, p_valor, _, _ = stats.chi2_contingency(tabela)
    diferenca = taxa2 - taxa1
    lift = (diferenca / taxa1 * 100) if taxa1 > 0 else 0

In [13]:
for grupo in dados.index:
    taxa = dados.loc[grupo, 'taxa']
    sucesso = "Bom" if taxa > 0.05 else "Médio" if taxa > 0.02 else "Ruim"
    print(f"{grupo}: {sucesso} ({taxa:.2%})")

if len(grupos) >= 2:
    print(f"\n{grupo1}: {taxa1:.2%}")
    print(f"{grupo2}: {taxa2:.2%}")
    print(f"Diferença: {diferenca:+.2%}")

    if diferenca > 0:
        print(f"{grupo2} é melhor")
    elif diferenca < 0:
        print(f"{grupo1} é melhor")
    else:
        print("Sem diferença")

    if p_valor < 0.05:
        print("Resultado confiável!")
        if diferenca > 0:
            print(f"Use {grupo2}! Ganha {lift:+.1f}%")
        elif diferenca < 0:
            print(f"Fique com {grupo1}!")
    else:
        print("Não é confiável. Teste mais!")

ad: Médio (2.55%)
psa: Ruim (1.79%)

ad: 2.55%
psa: 1.79%
Diferença: -0.77%
ad é melhor
Resultado confiável!
Fique com ad!
